# BERT for Multi-Label Classification

In [1]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

/home/lipsa/anaconda3/envs/iia/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lipsa/anaconda3/envs/iia/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lipsa/anaconda3/envs/iia/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lipsa/anaconda3/envs/iia/lib/python3.6/site-packages

In [2]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [3]:
##use downloaded model, change path accordingly
BERT_VOCAB= '/home/lipsa/Downloads/LIPSA APP/uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '/home/lipsa/Downloads/LIPSA APP/uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '/home/lipsa/Downloads/LIPSA APP/uncased_L-12_H-768_A-12/bert_config.json'

In [4]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [5]:

tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

# dataset

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
df=pd.read_csv("annotated_iia_skype_dataset_for_LIPSA_APP.csv")
df=df.append(pd.read_csv("annotated_genarated_data_Casual&Deadline.csv"))

In [8]:
import sklearn.utils
new= sklearn.utils.shuffle(df)

In [12]:
new=new.reset_index(drop=1)
new

,Dialogue,Decision,Deadline,Casual,Discussion,Report
0,Alternative UX solution proposition for Multip...,0,0,0,1,1
1,All scenes - Bug fixes: In Progress 50 %,0,0,0,0,1
2,Pipeline compare result for web (not,0,0,0,1,1
3,Signed and Forgot password - Donee (Mobile and W,0,0,0,0,1
4,Iterations / finalization of October release p...,0,0,0,1,1
...,...,...,...,...,...,...
7050,Web went live on Oct 8th night – but Mobile (e...,0,0,0,1,1
7051,", for thursday as far we hate this, a client i...",0,1,0,0,0
7052,legacy - 5 centers payment posting trial run d...,0,1,0,0,0
7053,Rolling out a support matrix for Art tor,1,0,0,1,0


In [11]:
new=new.drop(['Unnamed: 0'],axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [13]:
#train_data=pd.DataFrame()
#train_data=train_data.append(new,ignore_index=True)
#train,test=train_test_split(train_data,test_size = 0.40, random_state = 0
train=pd.DataFrame()
train=train.append(new,ignore_index=True)
train.to_csv('Bert_Train.csv')
#test.to_csv('Bert_Test.csv')

In [14]:
##change path accordingly
#train_data_path='./Downloads/train.csv'
train = pd.read_csv('Bert_Train.csv')

test = pd.read_csv('Bert_Test.csv')

In [15]:
train=train.drop(['Unnamed: 0'],axis=1)
train=train.reset_index()

In [16]:
train.columns

Index(['index', 'Dialogue', 'Decision', 'Deadline', 'Casual', 'Discussion',
       'Report'],
      dtype='object')

In [19]:
train.head()

,index,Dialogue,Decision,Deadline,Casual,Discussion,Report
0,0,Alternative UX solution proposition for Multip...,0,0,0,1,1
1,1,All scenes - Bug fixes: In Progress 50 %,0,0,0,0,1
2,2,Pipeline compare result for web (not,0,0,0,1,1
3,3,Signed and Forgot password - Donee (Mobile and W,0,0,0,0,1
4,4,Iterations / finalization of October release p...,0,0,0,1,1


In [16]:
test_cm=test
test=test.drop(['Unnamed: 0','Discussion', 'Decision','Deadline', 'Report', 'Casual'],axis=1)
test=test.reset_index()

In [17]:
test.head()

,index,Dialogue
0,0,Add family work flow screens (prototyping
1,1,and - - meet 30
2,2,"My favorite is Rock and Roll music; however, I..."
3,3,2w ai demo
4,4,"Show a small quot; New Posts """


In [20]:
len(train),len(test)

(7055, 2822)

In [21]:
ID = 'index'
DATA_COLUMN = 'Dialogue'
LABEL_COLUMNS = ['Decision', 'Deadline', 'Casual', 'Discussion','Report']

In [22]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [23]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[2:]
        else:
            labels = [0,0,0,0,0]
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [24]:
TRAIN_VAL_RATIO = 0.9
LEN = train.shape[0]
SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)

x_train = train[:SIZE_TRAIN]
x_val = train[SIZE_TRAIN:]

train_examples = create_examples(x_train)

In [25]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [26]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

In [27]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 10
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [28]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """
    
    
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([5], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [29]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [56]:
num_train_steps,num_warmup_steps

(634, 63)

In [30]:
train_file = os.path.join('/home/lipsa/Downloads/LIPSA APP', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

In [31]:
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 6349
INFO:tensorflow:  Batch size = 10
INFO:tensorflow:  Num steps = 634


In [32]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

In [33]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [34]:
OUTPUT_DIR = "/home/lipsa/Downloads/LIPSA APP/output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [35]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/home/lipsa/Downloads/LIPSA APP/output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f27c6429e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate sho

INFO:tensorflow:num_labels:5;logits:Tensor("loss/BiasAdd:0", shape=(10, 5), dtype=float32);labels:Tensor("loss/Cast:0", shape=(10, 5), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:**** Trainable Variables ****


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.



INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/lipsa/Downloads/LIPSA APP/output/model.ckpt-444
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 444 into /home/lipsa/Downloads/LIPSA APP/output/model.ckpt.
INFO:tensorflow:loss = 0.56781584, step = 445
INFO:tensorflow:global_step/sec: 0.09852
INFO:tensorflow:loss = 0.48732147, step = 545 (1015.023 sec)
INFO:tensorflow:Saving checkpoints for 634 into /home/lipsa/Downloads/LIPSA APP/output/model.ckpt.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:Loss for final step: 0.3439925.
Training took time  0:32:13.393252


In [37]:
eval_file = os.path.join('/home/lipsa/Downloads/LIPSA APP', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [38]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:num_labels:5;logits:Tensor("loss/BiasAdd:0", shape=(?, 5), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 5), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-29T12:44:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/lipsa/Downloads/LIPSA APP/output/model.ckpt-634
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-04-29-12:48:27
INFO:tensorflow:Saving dict for global step 634: 0 = 0.7682427, 1 = 0.9525877, 2 = 0.91183525, 3 = 0.7620257, 4 = 0.869972, eval_loss = 0.40062287, global_step = 634, loss = 0.40181816
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 634: /home/lipsa/Downloads/LIPSA APP/output/model.ckpt-634


In [37]:
output_eval_file = os.path.join("/home/lipsa/Downloads/LIPSA APP", "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  0 = 0.7321101
INFO:tensorflow:  1 = 0.9074388
INFO:tensorflow:  2 = 0.92076796
INFO:tensorflow:  3 = 0.7572938
INFO:tensorflow:  4 = 0.8329949
INFO:tensorflow:  eval_loss = 0.45976374
INFO:tensorflow:  global_step = 444
INFO:tensorflow:  loss = 0.4607851


In [3]:
import pandas as pd

In [4]:
test=pd.read_csv("Totest_vidya_conversation.csv")
#test=test.drop(['Unnamed: 0'],axis=1)
test=test.reset_index()
test

,index,Dialogue
0,0,I think it is time that we decide on topic for...
1,1,Mausam sir expects a paper from the batch
2,2,You are right
3,3,I am too confused
4,4,How can we read a paper code it and improve th...
5,5,This seems too much to ask for
6,6,o not worry
7,7,We can look for code of a paper online and dir...
8,8,I think finding a paper with code as well as d...
9,9,Why do not I try something on sentiment analysis


In [41]:
x_test = test[:10000] #testing a small sample
x_test = x_test.reset_index(drop=True)
predict_examples = create_examples(x_test,False)

In [42]:
test_features = convert_examples_to_features(predict_examples, MAX_SEQ_LENGTH, tokenizer)

I think it is time that we decide on topic for the NLP project
Mausam sir expects a paper from the batch
You are right
I am too confused
How can we read a paper code it and improve the baseline within a month
This seems too much to ask for
o not worry
We can look for code of a paper online and directly jump on improving baseline
I think finding a paper with code as well as dataset available will be a challenge
Why do not I try something on sentiment analysis
that is seem cool
But I beleive working on things like Sentiment analysis will be boring considering we already hace done an assignment on it
Are you familiar with generative language models
I too find them interesting
this is gonna be tough
I think this will require a lot of hardwork
Given that this semester is fully loaded why do you want to try such a new thing
I too believe generative models are tough
They will be difficult to train and implement
Trying new things is what we are here for
Challenges will always be there but we m

In [43]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [44]:
print('Beginning Predictions!')
current_time = datetime.now()

predict_input_fn = input_fn_builder(features=test_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
predictions = estimator.predict(predict_input_fn)
print("Prediction took time ", datetime.now() - current_time)

Beginning Predictions!
Prediction took time  0:00:00.000261


In [45]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    
    return dff
        

In [46]:
output_df = create_output(predictions)
test_data_result=  pd.concat([x_test, output_df], axis=1)
submission = test_data_result.drop(['Dialogue'], axis=1)
submission.to_csv("sample_submission0.csv", index=False)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:num_labels:5;logits:Tensor("loss/BiasAdd:0", shape=(?, 5), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 5), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 5), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/lipsa/Downloads/LIPSA APP/output/model.ckpt-634
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [47]:
test_data_result


,index,Dialogue,Decision,Deadline,Casual,Discussion,Report
0,0,I think it is time that we decide on topic for...,0.126831,0.249124,0.413240,0.177134,0.123767
1,1,Mausam sir expects a paper from the batch,0.112283,0.379711,0.323056,0.163872,0.169528
2,2,You are right,0.119728,0.156289,0.760034,0.117410,0.149117
3,3,I am too confused,0.125734,0.151243,0.778395,0.128750,0.134068
4,4,How can we read a paper code it and improve th...,0.363603,0.253287,0.182480,0.375795,0.208027
5,5,This seems too much to ask for,0.149243,0.108928,0.768394,0.145776,0.142082
6,6,o not worry,0.142711,0.154575,0.710379,0.130435,0.135077
7,7,We can look for code of a paper online and dir...,0.550978,0.129211,0.066333,0.603227,0.352432
8,8,I think finding a paper with code as well as d...,0.194764,0.053380,0.199108,0.267552,0.200488
9,9,Why do not I try something on sentiment analysis,0.214907,0.173050,0.236072,0.277440,0.178643


In [51]:
ne=submission.values.tolist()
for i in range(len(ne)):
    del ne[i][0]
    nn=[1 if j>0.45 else 0 for j in ne[i]]
    ne[i]=nn
#testne=test_cm.values.tolist()
#for i in range(len(testne)):
 #   del testne[i][0]
   # del testne[i][0]

In [54]:
dictlabel={'Decision':[], 'Deadline':[], 'Casual':[],'Discussion':[],'Report':[]}
for pos in ne:
    for index,i in enumerate(dictlabel.keys()):
        dictlabel[i].append(pos[index])
test_data_result=test_data_result.drop(['Decision', 'Deadline', 'Casual','Discussion','Report'],axis=1)
for i in dictlabel.keys():
    test_data_result[i]=dictlabel[i]
result=test_data_result
result
newdf=result.drop(['index','Dialogue'],axis=1)
data=newdf.values.tolist()
original=[]
for index,i in enumerate(data):
    label=[]
    if i[0]==1:
        label.append('Decision')
    if i[1]==1:
        label.append('Deadline')
    if i[2]==1:
        label.append('Casual')
    if i[3]==1:
        label.append('Discussion')
    if i[4]==1:
        label.append('Report')
    original.append(label)
newresult=test_data_result.drop(['index','Decision', 'Deadline', 'Casual', 'Discussion','Report'],axis=1)
newresult['Label']=original
newresult.to_csv("To_test_answer.csv")

In [57]:
newresult

,Dialogue,Label
0,I think it is time that we decide on topic for...,[]
1,Mausam sir expects a paper from the batch,[]
2,You are right,[Casual]
3,I am too confused,[Casual]
4,How can we read a paper code it and improve th...,[]
5,This seems too much to ask for,[Casual]
6,o not worry,[Casual]
7,We can look for code of a paper online and dir...,"[Decision, Discussion]"
8,I think finding a paper with code as well as d...,[]
9,Why do not I try something on sentiment analysis,[]


##### reference

https://github.com/google-research/bert/blob/master/run_classifier.py

https://github.com/yajian/bert/blob/master/run_multilabels_classifier.py

##### medium post
https://towardsdatascience.com/building-a-multi-label-text-classifier-using-bert-and-tensorflow-f188e0ecdc5d

In [69]:
ne=submission.values.tolist()

In [70]:
for i in range(len(ne)):
    del ne[i][0]
    #f,s=largest_second(ne[i])
    #ne[i][ne[i].index(f)]=1
    #ne[i][ne[i].index(s)]=1
    nn=[1 if j>0.4 else 0 for j in ne[i]]
    ne[i]=nn

In [71]:
def largest_second(list1):   
    largest = list1[0]  
    largest2 = None
    for item in list1[1:]:      
        if item > largest:  
            largest2 = largest 
            largest = item  
        elif largest2 == None or largest2 < item:  
            largest2 = item  
    return largest,largest2
  
  

In [72]:
dictlabel={'Decision':[], 'Deadline':[], 'Casual':[],'Discussion':[],'Report':[]}
for pos in ne:
    for index,i in enumerate(dictlabel.keys()):
        dictlabel[i].append(pos[index])
test_data_result=test_data_result.drop(['Decision', 'Deadline', 'Casual','Discussion','Report'],axis=1)
for i in dictlabel.keys():
    test_data_result[i]=dictlabel[i]


In [73]:
testne=test_cm.values.tolist()

In [76]:
testne

[['A demo are scheduled for Malaysian claim', 1, 0, 0, 1, 1],
 ['Verify & collect proper data for styleAssis', 0, 0, 0, 1, 1],
 ['stylish and trend based mapping recommendation to people 10th october end',
  0,
  1,
  0,
  0,
  0],
 ['Then, I can give them a little of coaching if the need arises.',
  0,
  0,
  1,
  0,
  0],
 ['New hand off implementation in all RASA bots', 1, 0, 0, 0, 1],
 ['Make sure you report in this group, if anyone from your team comes up with similar symptoms',
  0,
  0,
  0,
  1,
  1],
 ['New Symbol system Remain and Support Automation team in srilank',
  0,
  0,
  0,
  1,
  1],
 ['Fix affiliate marketing link tracking for IOS / Apple app store downloads',
  1,
  0,
  0,
  1,
  0],
 ['And we shouldnt be treating different generations same way', 0, 0, 1, 0, 0],
 ['recommendation to make deliver here for on priority tasks for spring 30',
  0,
  1,
  0,
  0,
  0],
 ['Review AR capabilities and possibilities for a POC for Art tor',
  1,
  0,
  0,
  0,
  0],
 ["That'

In [77]:
for i in range(len(testne)):
    del testne[i][0]
    #del testne[i][0]

In [78]:
import numpy as np


In [79]:
y_true=np.array(testne)

In [80]:
y_pred=np.array(ne)

In [81]:
from sklearn.metrics import confusion_matrix
labels = ['Decision', 'Deadline', 'Casual', 'Discussion','Report']

conf_mat_dict={}

for label_col in range(len(labels)):
    y_true_label = y_true[:, label_col]
    y_pred_label = y_pred[:, label_col]
    conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)


for label, matrix in conf_mat_dict.items():
    print("Confusion matrix for label {}:".format(label))
    print(matrix)

Confusion matrix for label Decision:
[[1183  154]
 [ 381  183]]
Confusion matrix for label Deadline:
[[1370   53]
 [ 220  258]]
Confusion matrix for label Casual:
[[1459  221]
 [  67  154]]
Confusion matrix for label Discussion:
[[545 601]
 [130 625]]
Confusion matrix for label Report:
[[682 356]
 [153 710]]


In [82]:
s=np.array(['Decision', 'Deadline', 'Casual', 'Discussion','Report'])

In [83]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_true, y_pred, sample_weight=None, labels=None, samplewise=False)

array([[[1183,  154],
        [ 381,  183]],

       [[1370,   53],
        [ 220,  258]],

       [[1459,  221],
        [  67,  154]],

       [[ 545,  601],
        [ 130,  625]],

       [[ 682,  356],
        [ 153,  710]]])

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true,y_pred)

0.25302472382956337

In [85]:
from sklearn.metrics import classification_report 
print (classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.32      0.41       564
           1       0.83      0.54      0.65       478
           2       0.41      0.70      0.52       221
           3       0.51      0.83      0.63       755
           4       0.67      0.82      0.74       863

   micro avg       0.58      0.67      0.62      2881
   macro avg       0.59      0.64      0.59      2881
weighted avg       0.61      0.67      0.61      2881
 samples avg       0.55      0.68      0.58      2881



/home/lipsa/anaconda3/envs/iia/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
